# Teaching a Turkish language model new tricks

Instead of creating a new Machine Learning (ML) model for every new task, we can leverage the concept of *Transfer Learning*. 

In particular, we can use generic language models and teach it new tasks by fine-tuning them using corresponding datasets. In this notebook we will use a Turkish language model created by the MDZ Digital Library team (dbmdz) at the Bavarian State Library (https://github.com/stefan-it/turkish-bert). We will use the Hugging Face Model Hub to download the model (https://huggingface.co/dbmdz/bert-base-turkish-uncased) and then fine-tune it to 2 very different tasks:

- Sentiment Analysis
- Question Answering

## SageMaker Setup

In [2]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20211005T160629
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


In [1]:
tokenizer_name = 'dbmdz/bert-base-turkish-uncased'
s3_prefix = 'samples/datasets/turkish_product_reviews'

## Sentiment Analysis

### Inspect the data

Taken from https://huggingface.co/datasets/turkish_product_reviews

In [3]:
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'

In [4]:
import botocore
from datasets.filesystems import S3FileSystem
from datasets import load_from_disk

s3_session = botocore.session.Session()
s3 = S3FileSystem(session=s3_session)

In [5]:
dataset = load_from_disk(training_input_path, fs=s3)

In [7]:
import pandas as pd
df_train = pd.DataFrame(dataset)

In [18]:
pd.set_option('display.max_colwidth', 0)

In [19]:
df_train[['sentence', 'labels']].iloc[15:20]

,sentence,labels
15,herkese tavsiye ederim.,1
16,"isınması geç, ızgara için iki tarafı da kullanıldığında arkadaki ızgarayı kapatırken içindeki yağlar aradan bağlantı kablosuna akıyor. izgara sonrası temizlemek çok zor. yanında verilen temizleme aparatı çok da işe yaramıyor. oluşan yağın akması için yapılan yere yağ akmıyor.",0
17,ürün anlatıldığı gibi. saçı yumuşatıyor kolay hacim almasını sağlıyor ve parlatıyor ama parlaklığı uzun sürmüyor,1
18,tablonun zemin rengi fotoğraflarda göründüğünden çok daha koyu bir yeşil..,0
19,"su geçirmez,desenli alezli visko yatak.bana gelen üründe koku yok.fotoğraftaki ürün birebir. hızlı kargo.fiyatının karşılığını bulabileceğiniz kaliteli bir yatak.simdiden hayırlı olsun bebelerimize :)",1


### Model Training

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 8,
                 'model_name': tokenizer_name
                 }

In [ ]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            hyperparameters = hyperparameters)

In [ ]:
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

In [ ]:
from sagemaker.estimator import Estimator
huggingface_estimator = Estimator.attach("huggingface-pytorch-training-2021-10-01-16-52-39-333")

### Model Deployment

In [30]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model_sentiment = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-10-01-16-52-39-333/output/model.tar.gz",
   role=role, # 
   transformers_version="4.6", 
   pytorch_version="1.7", 
   py_version="py36", 
)

In [31]:
predictor = huggingface_model_sentiment.deploy(1, "ml.m5.xlarge")

----------------!

### Model Testing

In [32]:
sentiment_input= {"inputs": "Bu oldukça kötü bir ürün, bunu kimseye tavsiye etmem"}
predictor.predict(sentiment_input)

[{'label': 'LABEL_0', 'score': 0.9829983711242676}]

In [42]:
sentiment_input= {"inputs": "Bu şampuanı seviyorum, saçlarımı çok parlak yapıyor"}
predictor.predict(sentiment_input)

[{'label': 'LABEL_1', 'score': 0.9976404905319214}]

## Question Answering

### Inspecting the data

Taken from https://github.com/TQuad/turkish-nlp-qa-dataset

In [22]:
data_files = {}
data_files["train"] = 'data/train.json'
data_files["validation"] = 'data/val.json'

In [25]:
from datasets import load_dataset
ds = load_dataset("json", data_files=data_files)

Using custom data configuration default-8dc8d5e84c5485b6
Reusing dataset json (/home/ec2-user/.cache/huggingface/datasets/json/default-8dc8d5e84c5485b6/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [26]:
df = pd.DataFrame(ds['train'])

In [29]:
df.sample(3)

,id,title,context,question,answers
2531,2729,Huneyn bin İshak,"Huneyn bin İshak (d. 810- ö. 873) Abbasi döneminde yaşamış ünlü bir mütercim ve hekimdir. Tam künyesi ""Ebu Zeyd Huneyn bin İshak el-İbadi'""dir. Hristiyan bir Arap aşireti olan İbad'a mensuptur . Arapça, Süryanice, Grekçe'yi çok iyi derecede bilen Huneyn, bu, dillerdeki yetkinliği ve tercümelerinin kalitesi sebebiyle Abbasi halifesi Memun döneminden El-Mutasım dönemine kadar Bağdat'a hekimlik ve mütercimlik yapmıştır . Pehlevice bildiği rivayet edilse de ders aldığı kişi, Halil bin Ahmed (ö. 791), o dünyaya gelmeden ölmüştür . İki oğlu vardır: Birinin adı Davut'tur ve çağında yetkin bir pratisyen hekim olarak tanınmıştır. Diğerinin adı ise İshak'tır ve İshak, babasından bile yetenekli tercüman olarak tanınmıştır.",Huneyn bin İshak’ın tam adı nedir?,"{'text': ['Ebu Zeyd Huneyn bin İshak el-İbadi'], 'answer_start': [103]}"
5654,6382,Baykar Makina,"Bayraktar Taktik İHA Sistemi, 17 Aralık 2015 tarihinde 2 adet Roketsan MAM-L güdümlü füzesi yüklü şekilde yapılan atış testinde %100 isabet ile hedefleri vurma başarısını göstermiştir. Bu deneme ile keşif ve gözetleme yaparken, anlık olarak ortaya çıkan hedefleri vurabileceğini gösterdi",Roketsan MAM-L güdümlü füzesi yüklü şekilde yapılan atış testinde ne kadar başarı sağlamıştır?,"{'text': ['%100'], 'answer_start': [128]}"
2625,2849,İsenbike Togan,"1990 yılında 'UNESCO Orta Asya Uygarlıkları Tarihini Yazma Komitesi'nin (UNESCO International Scientific Committee for the drafting of a History of Civilisations of Central Asia) bir üyesi olmuş ve aynı dönemde UNESCO İpek Yolu Araştırmaları (UNESCO Integral Study of the Silk Roads) projesi ile 1990'da Çin Halk Cumhuriyeti ve 1991'de Özbekistan, Tacikistan, Kazakistan ve Kırgızistan'da araştırmalar yapmıştır. 1992'de Türkiye'ye dönmüş ve doçentliğini alarak, ODTÜ Tarih Bölümü'nde öğretim üyeliği yapmaya devam etmiştir. 1995 yılında da Profesör olmuştur. 2006 yılında ODTÜ Tarih Bölümü'nden emekli olmuş ancak günümüzde Boğaziçi Üniversitesi'nde Çin tarihi dersleri vermeye devam etmektedir. Ayrıca Türk Tarih Kurumu, 'Türk-Rus Tarihçileri Karma Komisyonu' üyesidir.",İsenbike Togan hangi yılda emekli olmuştur?,"{'text': ['2006 yılında '], 'answer_start': [560]}"


### Model Training

In [ ]:
from sagemaker.huggingface import HuggingFace

hyperparameters={
    'model_name_or_path': 'dbmdz/bert-base-turkish-uncased',
    'train_file': '/opt/ml/input/data/train/train.json',
    'validation_file': '/opt/ml/input/data/val/val.json',
    'do_train': True,
    'do_eval': False,
    'fp16': True,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'num_train_epochs': 2,
    'max_seq_length': 384,
    'pad_to_max_length': True,
    'doc_stride': 128,
    'output_dir': '/opt/ml/model'
}

instance_type = 'ml.p3.16xlarge'
instance_count = 1
volume_size = 200

In [ ]:
huggingface_estimator = HuggingFace(entry_point='run_qa.py',
                                    source_dir='./scripts',
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    role=role,
                                    transformers_version='4.10',
                                    pytorch_version='1.9',
                                    py_version='py38',
                                    hyperparameters= hyperparameters,
                                    disable_profiler=True,
                                   )

In [ ]:
huggingface_estimator.fit({'train': 's3://sagemaker-us-east-1-905847418383/qa-tr/data/', 'val': 's3://sagemaker-us-east-1-905847418383/qa-tr/data/'})

### Model Deployment

In [36]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model_qa = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-905847418383/huggingface-pytorch-training-2021-10-12-05-07-28-087/output/model.tar.gz",
   role=role, # 
   transformers_version="4.10", 
   pytorch_version="1.9", 
   py_version="py38", 
)

In [37]:
predictor_qa = huggingface_model_qa.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

### Model Testing

In [41]:
data = {
"inputs": {
    "question": "Ne zaman avare bir hayata başladı?",
    "context": """ABASIYANIK, Sait Faik. Hikayeci (Adapazarı 23 Kasım 1906-İstanbul 11 Mayıs 1954). \
İlk öğrenimine Adapazarı’nda Rehber-i Terakki Mektebi’nde başladı. İki yıl kadar Adapazarı İdadisi’nde okudu.\
İstanbul Erkek Lisesi’nde devam ettiği orta öğrenimini Bursa Lisesi’nde tamamladı (1928). İstanbul Edebiyat \
Fakültesi’ne iki yıl devam ettikten sonra babasının isteği üzerine iktisat öğrenimi için İsviçre’ye gitti. \
Kısa süre sonra iktisat öğrenimini bırakarak Lozan’dan Grenoble’a geçti. Üç yıl başıboş bir edebiyat öğrenimi \
gördükten sonra babası tarafından geri çağrıldı (1933). Bir müddet Halıcıoğlu Ermeni Yetim Mektebi'nde Türkçe \
gurup dersleri öğretmenliği yaptı. Ticarete atıldıysa da tutunamadı. Bir ay Haber gazetesinde adliye muhabirliği\
yaptı (1942). Babasının ölümü üzerine aileden kalan emlakin geliri ile avare bir hayata başladı. Evlenemedi.\
Yazları Burgaz adasındaki köşklerinde, kışları Şişli’deki apartmanlarında annesi ile beraber geçen bu fazla \
içkili bohem hayatı ömrünün sonuna kadar sürdü."""
    }
}
predictor_qa.predict(data)['answer']

'Babasının ölümü üzerine'